In [1]:
import numpy as np
from pynld.ds import DynamicalSystem, IntegrationParameters
from rich.console import Console
from rich.traceback import install
from bokeh.plotting import figure, show, output_notebook 
from bokeh.models import HoverTool, ColumnDataSource
from tqdm import tqdm
from scipy.special import jn

console = Console()
install()

<bound method InteractiveShell.excepthook of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x106386cf0>>

In [2]:
def autonomous_system(t, state_vector, params):
    psi1, psi3 = state_vector
    omega, gamma, kappa, eps, Omega, L1, L3 = params
    epsc = eps/np.cos(gamma)

    # forcing
    L = L1*np.sin(psi1) + L3*np.sin(psi3)

    psi1_dot = omega - Omega + epsc*L
    psi3_dot = omega - 3*Omega + epsc*L

    return np.array([psi1_dot, psi3_dot], np.float64)

In [3]:
def reduced_system(t, state_vector, params):
    psi1 = t
    psi3 = state_vector
    omega, gamma, kappa, eps, Omega, L1, L3 = params
    epsc = eps/np.cos(gamma)

    L = L1*np.sin(psi1) + L3*np.sin(psi3)
    psi3_dot = (omega - 3*Omega + epsc*L)/(omega - Omega + epsc*L)

    return np.array([psi3_dot])

In [4]:
def test_system(t, state_vector, params):
    eps = params
    x = state_vector

    dxdt = 1 + eps * np.sin(x)

    return np.array([dxdt])

In [63]:
p = {
    'omega':    1.0,
    'gamma':    0.1,
    'kappa':    -2.0,
    'eps':      0.05,
    'Omega':    0.332,
    'L1':       1.00,
    'L3':       0.10
}

p_test = {
    'eps': 0.1,
}

# Set initial state and time span
#u0 = {'x': 1.0, 'y': 0.0, 'phi1': 0.0, 'phi2': 0.0}
#u0 = {'psi1': 0.0, 'psi3': 0.0}
u0 = {'psi3': 0.0}

In [64]:
ip = IntegrationParameters(solver='RK45', time_step=1e-3)

#phs = DynamicalSystem(autonomous_system, t0=0, x0=u0, parameters=p, 
#                      integration_params=ip)

phs = DynamicalSystem(reduced_system, t0=0, x0=u0, parameters=p, 
                      integration_params=ip)

In [65]:
def poincare_map(x, y, cycle):
    p_map = []
    N = 1
    for i in range(len(x)):
        if x[i] > N * cycle:
            p_map.append(y[i])
            N += 1
    
    return np.asarray(p_map)

In [71]:
nu = (p['omega'] - 3*p['Omega'])/(p['omega'] - p['Omega'])
eps = p['eps']/((p['omega'] - p['Omega'])*np.cos(p['gamma']))
C0 = 0
C1 = (2*(1-nu)/nu)*(nu*p['L1']+p['L3'])
C2 = 0

def y0(x):
    return nu * x + C0

def y1(x):
    return C1 - 2*(1-nu)*(p['L1']*np.cos(x)+(p['L3']/nu)*np.cos(nu*x+C0))

def y2(x):
    A = 2*(p['L1']**2 + p['L3']**2)*nu*x
    B = p['L1']**2*nu*np.sin(2*x) + p['L3']**2*np.sin(2*(C0+nu*x))
    C = 8*p['L1']*p['L3']*nu*(np.sin(x)*np.cos(C0+nu*x)+nu*np.sin(C0+nu*x)*np.cos(x))
    return C2 - ((1-nu)/(2*nu))*(A-B-C)

def y(x):
    return y0(x) + eps*y1(x) + eps**2*y2(x)

In [67]:
tr_time, run_time = 0, 10000

#phs.set_parameter('Omega', 0.30)
phs.reset()
phs.integrate_local(run_time, tr_time)

In [72]:
plt = figure(title="Phase portrait", 
               x_axis_label="psi_1",
               y_axis_label="psi_3",
               width=1200,
               height=600)
sparcity = 1000
output_notebook()
plt.scatter(phs.t_sol[::sparcity], 
            phs.x_sol[0,::sparcity],
            #legend_label="Full XY",
            size=2, color='#344966', alpha=0.5)
        
plt.scatter(phs.t_sol[::sparcity],
         y(phs.t_sol[::sparcity]),
         size=1, color='#FF6666')

#plt.line(np.mod(phs.t_sol[::sparcity], 2*np.pi), 
#         phs.x_sol[0,::sparcity],
#         legend_label="Full XY",
#         line_width=1, color='#FF6666')

plt.add_tools(HoverTool(tooltips=[("Omega", "@x"), ("Ratio", "@y")]))
#plt.legend.location = 'top_left'
show(plt)

Loading BokehJS ...

In [29]:
psi3_map = poincare_map(phs.t_sol, phs.x_sol[0], 2*np.pi)

plt = figure(title="Poincare map", 
               x_axis_label="n",
               y_axis_label="psi_3",
               width=1200,
               height=600)
sparcity = 1000
output_notebook()
plt.scatter(np.arange(0, len(psi3_map)), 
            psi3_map,
            #legend_label="Full XY",
            size=3, color='#344966', alpha=0.8)

#plt.line(np.mod(phs.t_sol[::sparcity], 2*np.pi), 
#         phs.x_sol[0,::sparcity],
#         legend_label="Full XY",
#         line_width=1, color='#344966')

plt.add_tools(HoverTool(tooltips=[("Omega", "@x"), ("Ratio", "@y")]))
#plt.legend.location = 'top_left'
show(plt)

Loading BokehJS ...